# Question 1

Import the packages we need for this question.

In [14]:
import numpy as np
from scipy.stats import norm
%pip install tabulate
from tabulate import tabulate


Note: you may need to restart the kernel to use updated packages.


Based on Course Notes Section 3.5, we generate the vectorized code for pricing European/American options of both put and call types using a binomial lattice.

In [15]:
def option_value(S0, K, T, r, sigma, CorP, EorA, Nsteps):
    # S0: current stock price
    # K: strike
    # T: expiry time
    # r: interest rate
    # sigma: volatility
    # CorP: True for a call option, False for put option
    # EorA: True for a European option, False for an American option
    # Nsteps: number of timesteps

    # timestep size
    delt = T/Nsteps

    # tree parameters
    u = np.exp(sigma * np.sqrt(delt))
    d = 1./u
    a = np.exp(r*delt)
    p = (a-d)/(u-d)
    
    # payoff at t=T
    W = S0*d**np.arange(Nsteps,-1,-1)*u**np.arange(0,Nsteps+1,1) # W is column vector of size Nsteps+1 × 1
    if CorP == True:
        W = np.maximum(W - K, 0)
    else:
        W = np.maximum(K - W, 0)
    
    # European backward recursion
    if EorA == True:
        for i in np.arange(Nsteps,0,-1):
            W = np.exp(-r*delt)*(p*W[1:i+1]+(1-p)*W[0:i])
        return W[0]
    
    # American backward recursion
    else:
        for i in np.arange(Nsteps,0,-1):
            
            # payoff at t=T
            W1 = S0*d**np.arange(i-1,-1,-1)*u**np.arange(0,i,1)
            if CorP == True:
                W1 = np.maximum(W1 - K, 0)
            else:
                W1 = np.maximum(K - W1, 0)
            
            W = np.maximum(np.exp(-r*delt)*(p*W[1:i+1]+(1-p)*W[0:i]),W1)
        return W[0]

Use the function `blsprice` provided with the assignment to generate the exact analytical solution values.

In [16]:
def blsprice(Price, Strike, Rate, Time, Volatility):
    sigma_sqrtT = Volatility * np.sqrt (Time)

    d1 = 1 / sigma_sqrtT * (np.log(Price / Strike) + (Rate + Volatility**2 / 2) * Time)
    d2 = d1 - sigma_sqrtT

    phi1 = norm.cdf(d1)
    phi2 = norm.cdf(d2)
    disc = np.exp (-Rate * Time)
    F    = Price * np.exp ((Rate) * Time)

    Call = disc * (F * phi1 - Strike * phi2)
    Put  = disc * (Strike * (1 - phi2) + F * (phi1 - 1))
    return Call, Put

In [17]:
sigma = 0.32
r = 0.04
T = 1
K = 100
S0 = 100

# European Call Option
CorP = True
EorA = True

Vtree1 = option_value(S0, K, T, r, sigma, CorP, EorA, 500)
Vtree2 = option_value(S0, K, T, r, sigma, CorP, EorA, 1000)
Vtree3 = option_value(S0, K, T, r, sigma, CorP, EorA, 2000)
Vtree4 = option_value(S0, K, T, r, sigma, CorP, EorA, 4000)
Vtree5 = option_value(S0, K, T, r, sigma, CorP, EorA, 8000)

Change1 = Vtree2 - Vtree1
Change2 = Vtree3 - Vtree2
Change3 = Vtree4 - Vtree3
Change4 = Vtree5 - Vtree4

Ratio1 = Change1/Change2
Ratio2 = Change2/Change3
Ratio3 = Change3/Change4

if CorP == True:
    Vexact = blsprice(S0,K,r,T,sigma)[0]
else:
    Vexact = blsprice(S0,K,r,T,sigma)[1]

table = {"Timestep": ["T/500","T/1000","T/2000","T/4000","T/8000","Exact Value"],
         "Tree Value": [Vtree1,Vtree2,Vtree3,Vtree4,Vtree5,Vexact],
         "Change": ["",Change1,Change2,Change3,Change4],
         "Ratio": ["","",Ratio1,Ratio2,Ratio3]}

print(tabulate(table,headers="keys",tablefmt="grid"))

+-------------+--------------+------------------------+--------------------+
| Timestep    |   Tree Value | Change                 | Ratio              |
+=============+==============+========================+====================+
| T/500       |      14.5133 |                        |                    |
+-------------+--------------+------------------------+--------------------+
| T/1000      |      14.5165 | 0.0031373723798235886  |                    |
+-------------+--------------+------------------------+--------------------+
| T/2000      |      14.518  | 0.0015689815869617263  | 1.999623453771049  |
+-------------+--------------+------------------------+--------------------+
| T/4000      |      14.5188 | 0.000784564475704741   | 1.9998121703794665 |
+-------------+--------------+------------------------+--------------------+
| T/8000      |      14.5192 | 0.00039230067854845174 | 1.9999059869274278 |
+-------------+--------------+------------------------+--------------------+

In [18]:
# European Put Option
CorP = False
EorA = True

Vtree1 = option_value(S0, K, T, r, sigma, CorP, EorA, 500)
Vtree2 = option_value(S0, K, T, r, sigma, CorP, EorA, 1000)
Vtree3 = option_value(S0, K, T, r, sigma, CorP, EorA, 2000)
Vtree4 = option_value(S0, K, T, r, sigma, CorP, EorA, 4000)
Vtree5 = option_value(S0, K, T, r, sigma, CorP, EorA, 8000)

Change1 = Vtree2 - Vtree1
Change2 = Vtree3 - Vtree2
Change3 = Vtree4 - Vtree3
Change4 = Vtree5 - Vtree4

Ratio1 = Change1/Change2
Ratio2 = Change2/Change3
Ratio3 = Change3/Change4

if CorP == True:
    Vexact = blsprice(S0,K,r,T,sigma)[0]
else:
    Vexact = blsprice(S0,K,r,T,sigma)[1]

table = {"Timestep": ["T/500","T/1000","T/2000","T/4000","T/8000","Exact Value"],
         "Tree Value": [Vtree1,Vtree2,Vtree3,Vtree4,Vtree5,Vexact],
         "Change": ["",Change1,Change2,Change3,Change4],
         "Ratio": ["","",Ratio1,Ratio2,Ratio3]}

print(tabulate(table,headers="keys",tablefmt="grid"))

+-------------+--------------+-----------------------+--------------------+
| Timestep    |   Tree Value | Change                | Ratio              |
+=============+==============+=======================+====================+
| T/500       |      10.5923 |                       |                    |
+-------------+--------------+-----------------------+--------------------+
| T/1000      |      10.5954 | 0.0031373723728744807 |                    |
+-------------+--------------+-----------------------+--------------------+
| T/2000      |      10.597  | 0.0015689816114807797 | 1.9996234180931405 |
+-------------+--------------+-----------------------+--------------------+
| T/4000      |      10.5978 | 0.0007845644998543122 | 1.999812140075326  |
+-------------+--------------+-----------------------+--------------------+
| T/8000      |      10.5981 | 0.0003923006295085685 | 1.9999062984862634 |
+-------------+--------------+-----------------------+--------------------+
| Exact Valu

The tree values for both European call and put options converge to the exact value.

By hand calculation and substituting $V_{\Delta t}^{tree}(S,t)=V^{exact}(S,t)+C(S,t)\Delta t+O((\Delta t)^2)$ into the formula, we estimate that

$$
\lim_{\Delta t \to 0} \frac{V_{\frac{\Delta t}{2}}^{tree}(S_0^0,0)-V_{\Delta t}^{tree}(S_0^0,0)}{V_{\frac{\Delta t}{4}}^{tree}(S_0^0,0)-V_{\frac{\Delta t}{2}}^{tree}(S_0^0,0)}
= \lim_{\Delta t \to 0} \frac{C(S,t)\frac{\Delta t}{2}-C(S,t)\Delta t}{C(S,t)\frac{\Delta t}{4}-C(S,t)\frac{\Delta t}{2}} = 2
$$

The convergence data generated indicates that the lattice method is likely exhibiting a first order of accuracy because the ratios for European call and put options both converge to $2$, which agrees with the theory above.

In [19]:
# American Put Option
CorP = False
EorA = False

Vtree1 = option_value(S0, K, T, r, sigma, CorP, EorA, 500)
Vtree2 = option_value(S0, K, T, r, sigma, CorP, EorA, 1000)
Vtree3 = option_value(S0, K, T, r, sigma, CorP, EorA, 2000)
Vtree4 = option_value(S0, K, T, r, sigma, CorP, EorA, 4000)
Vtree5 = option_value(S0, K, T, r, sigma, CorP, EorA, 8000)

Change1 = Vtree2 - Vtree1
Change2 = Vtree3 - Vtree2
Change3 = Vtree4 - Vtree3
Change4 = Vtree5 - Vtree4

Ratio1 = Change1/Change2
Ratio2 = Change2/Change3
Ratio3 = Change3/Change4

table = {"Timestep": ["T/500","T/1000","T/2000","T/4000","T/8000"],
         "Tree Value": [Vtree1,Vtree2,Vtree3,Vtree4,Vtree5],
         "Change": ["",Change1,Change2,Change3,Change4],
         "Ratio": ["","",Ratio1,Ratio2,Ratio3]}

print(tabulate(table,headers="keys",tablefmt="grid"))

+------------+--------------+------------------------+--------------------+
| Timestep   |   Tree Value | Change                 | Ratio              |
+============+==============+========================+====================+
| T/500      |      10.9918 |                        |                    |
+------------+--------------+------------------------+--------------------+
| T/1000     |      10.9935 | 0.00163496045544953    |                    |
+------------+--------------+------------------------+--------------------+
| T/2000     |      10.9943 | 0.0008220116075037254  | 1.988974900749343  |
+------------+--------------+------------------------+--------------------+
| T/4000     |      10.9947 | 0.0004041705718158539  | 2.0338235013266774 |
+------------+--------------+------------------------+--------------------+
| T/8000     |      10.9949 | 0.00020238367639890953 | 1.9970512395437028 |
+------------+--------------+------------------------+--------------------+


The tree values for American put options also converge to a specific value, but the ratios fluctuate around $2$ due to the early exercising property of the American put option.